In [2]:
#讀檔
import json

text = []
for i in range(65,78):
    range_2 = 10
    if i == 77:
        range_2 = 8
    for j in range(range_2):
        range_3 = 10
        if i == 77 and j == 7:
            range_3 = 4
        for l in range(range_3):
            path = 'wiki_zh/A' + chr(i) + '/wiki_' + str(j) + str(l)

            input_file = open(path, encoding = 'utf8')
            for line in input_file.readlines():
                text.append(json.loads(line))
            input_file.close()
            

In [3]:
#簡轉繁
from opencc import OpenCC

cc = OpenCC('s2t')

for item in text:
    item['title'] = cc.convert(item['title'])
    item['text'] = cc.convert(item['text'])

In [4]:
# 斷詞
import jieba

fileTrainSeg=[]
for i in text:
    fileTrainSeg.append([' '.join(list(jieba.cut(i['text'],cut_all=False)))])

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\UserTest\AppData\Local\Temp\jieba.cache
Loading model cost 1.948 seconds.
Prefix dict has been built successfully.


In [6]:
#斷詞結果寫檔
fileSegWordDonePath ='corpusSegDone.txt'
with open(fileSegWordDonePath,'wb') as fW:
    for i in range(len(fileTrainSeg)):
        fW.write(fileTrainSeg[i][0].encode('utf-8'))
        fW.write('\n'.encode('utf-8'))

In [5]:
# jieba分詞轉word2vec向量
import word2vec

word2vec.word2vec('corpusSegDone.txt', 'corpusWord2Vec.txt', size=300,verbose=True)

Running command: word2vec -train corpusSegDone.txt -output corpusWord2Vec.txt -size 300 -window 5 -sample 1e-3 -hs 0 -negative 5 -threads 12 -iter 5 -min-count 5 -alpha 0.025 -debug 2 -binary 0 -cbow 1
Starting training using file corpusSegDone.txt
Vocab size: 9233130KK1500KK800K500K900K
Words in train file: 244600906


Alpha: 0.014560  Progress: 41.76%  Words/thread/sec: 102.92k  Words/thread/sec: 10.18k  19.53k  sec: 38.45k  3.60k   .62%  Words/thread/sec: 57.21k  : 0.024839  Progress: 0.65%  Words/thread/sec: 59.16k  .90k  ad/sec: 76.80k  s: 1.29%  Words/thread/sec: 88.45k  s/thread/sec: 90.83k  : 1.40%  Words/thread/sec: 93.24k  Progress: 1.42%  Words/thread/sec: 93.97k    Words/thread/sec: 135.07k  .38k  thread/sec: 137.50k  .20%  Words/thread/sec: 138.00k    Progress: 3.26%  Words/thread/sec: 139.04k  d/sec: 141.41k  32k  %  Words/thread/sec: 143.68k  2k  ords/thread/sec: 149.10k    : 4.28%  Words/thread/sec: 152.75k   4.37%  Words/thread/sec: 153.63k   Words/thread/sec: 154.74k  : 155.00k  .49k  ords/thread/sec: 157.45k  .77k  5.08%  Words/thread/sec: 158.73k   Progress: 5.10%  Words/thread/sec: 158.71k  d/sec: 160.75k  023600  Progress: 5.61%  Words/thread/sec: 161.50k  3%  Words/thread/sec: 162.67k   163.06k    164.21k  0.023328  Progress: 6.69%  Words/thread/sec: 168.25k  3321  Progress: 6.7

Alpha: 0.000020  Progress: 99.92%  Words/thread/sec: 113.17k  46%  Words/thread/sec: 109.80k  32  Progress: 89.48%  Words/thread/sec: 109.80k   89.51%  Words/thread/sec: 109.79k  rogress: 89.89%  Words/thread/sec: 109.36k  Progress: 89.94%  Words/thread/sec: 109.40k   Words/thread/sec: 109.47k  sec: 109.49k  9.66k  hread/sec: 109.68k  lpha: 0.002290  Progress: 90.85%  Words/thread/sec: 109.88k  /thread/sec: 110.07k   Progress: 91.39%  Words/thread/sec: 110.08k  49%  Words/thread/sec: 110.15k  2113  Progress: 91.55%  Words/thread/sec: 110.20k  a: 0.002109  Progress: 91.57%  Words/thread/sec: 110.21k  hread/sec: 110.35k  10.70k  read/sec: 110.59k  sec: 110.64k  s: 93.70%  Words/thread/sec: 110.93k  527  Progress: 93.90%  Words/thread/sec: 111.05k  466  Progress: 94.14%  Words/thread/sec: 111.22k  gress: 94.39%  Words/thread/sec: 111.41k   Progress: 94.82%  Words/thread/sec: 111.70k  read/sec: 111.89k  .97k  : 0.001097  Progress: 95.61%  Words/thread/sec: 112.23k  001091  Progress: 95.64%

In [17]:
import word2vec

#驗證
model = word2vec.load('corpusWord2Vec.txt')